In [ ]:
import antigranular as ag
session = ag.login(<client_id>,<client_secret>, competition = "Fraud Detection Hackathon with Privacy Village #DPD25FEST")


Dataset "Transaction Fraud Hackathon Dataset" loaded to the kernel as transaction_fraud_hackathon_dataset
Key Name                       Value Type     
---------------------------------------------
test_x                         DataFrame      
train_x                        PrivateDataFrame
train_y                        PrivateDataFrame

Connected to Antigranular server session id: e54ad921-1d29-4447-80cd-36fa1509483b, the session will time out if idle for 25 minutes
Cell magic '%%ag' registered successfully, use `%%ag` in a notebook cell to execute your python code on Antigranular private python server
🚀 Everything's set up and ready to roll!


In [2]:
%%ag
x_train = transaction_fraud_hackathon_dataset["train_x"]
y_train = transaction_fraud_hackathon_dataset["train_y"]
x_test = transaction_fraud_hackathon_dataset["test_x"]

In [3]:
%%ag
from op_pandas import PrivateDataFrame
x_test=PrivateDataFrame(x_test)

In [4]:
%%ag
x_train_f=x_train.copy()
x_test_f=x_test.copy()
x_train_f[f"sum"]=x_train["a0"]+x_train["a1"]+x_train["a2"]+x_train["a3"]+x_train["a4"]+x_train["a5"]+x_train["a6"]+x_train["a7"]+x_train["a8"]
x_test_f[f"sum"]=x_test["a0"]+x_test["a1"]+x_test["a2"]+x_test["a3"]+x_test["a4"]+x_test["a5"]+x_test["a6"]+x_test["a7"]+x_test["a8"]

In [ ]:
%%ag
import tensorflow as tf
from op_pandas import standard_scaler, PrivateDataFrame
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from op_tensorflow import PrivateKerasModel, PrivateDataLoader
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, BatchNormalization

model = Sequential([
    # Input layer with 1024 neurons and GELU activation
    # Expects input shape of 10 features
    Dense(1024, activation='gelu', input_shape=(10,)),
    
    # Dropout layer to prevent overfitting (15% dropout rate)
    Dropout(0.15),
    
    # Batch normalization to stabilize learning
    BatchNormalization(),
    
    # Hidden layer with 512 neurons and GELU activation
    Dense(512, activation='gelu'),
    Dropout(0.1),
    BatchNormalization(),
    
    # Hidden layer with 256 neurons and GELU activation
    Dense(256, activation='gelu'),
    Dropout(0.05),
    BatchNormalization(),
    
    # Another hidden layer with 256 neurons and GELU activation
    Dense(256, activation='gelu'),
    Dropout(0.05),
    BatchNormalization(),
    
    # Hidden layer with 128 neurons and GELU activation
    Dense(128, activation='gelu'),
    Dropout(0.05),
    BatchNormalization(),
    
    # Output layer with sigmoid activation for binary classification
    Dense(1, activation='sigmoid')
])
# Create a differentially private model
dp_model = PrivateKerasModel(model=model, l2_norm_clip=1, noise_multiplier=0.5)

# Compile the model 
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

dp_model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=["accuracy"]
)


In [6]:
%%ag
data_loader = PrivateDataLoader(feature_df=x_train_f , label_df=y_train, batch_size=128)

In [ ]:
%%ag
dp_model.fit(x=data_loader, epochs=15)

In [ ]:
%%ag
def f(x: float) -> float:
    return 1 if x > 0.5 else 0

In [ ]:
%%ag
y_pred = dp_model.predict(x_test_f, label_columns=["output"])

In [ ]:
%%ag
y_pred["output"] = y_pred["output"].map(f, output_bounds=(0, 1))
result = submit_predictions(y_pred)